# Import

In [1]:
import tensorflow as tf
import numpy as np


# Data Loader
Use common tf.datasets.

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的圖片預設為uint8（0-255的數字）。以下程式碼將其正規化到0-1之間的浮點數，並在最後增加一維作為顏色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 從資料集中隨機取出batch_size個元素並返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

# Model Class

In [3]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten層將除第一維（batch_size）以外的維度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

# Hyperparameters

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

11493376/11490434 [==============================] - 2s 0us/step

# Training

In [6]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        if batch_index % 100 == 0:
            print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 0.024130
batch 100: loss 0.042086
batch 200: loss 0.055939
batch 300: loss 0.002223
batch 400: loss 0.029629
batch 500: loss 0.035381
batch 600: loss 0.008293
batch 700: loss 0.018466
batch 800: loss 0.086306
batch 900: loss 0.031112
batch 1000: loss 0.128564
batch 1100: loss 0.034869
batch 1200: loss 0.158145
batch 1300: loss 0.243777
batch 1400: loss 0.004737
batch 1500: loss 0.003788
batch 1600: loss 0.042852
batch 1700: loss 0.023309
batch 1800: loss 0.104312
batch 1900: loss 0.016270
batch 2000: loss 0.014091
batch 2100: loss 0.012818
batch 2200: loss 0.008531
batch 2300: loss 0.180443
batch 2400: loss 0.026535
batch 2500: loss 0.032802
batch 2600: loss 0.005621
batch 2700: loss 0.194394
batch 2800: loss 0.020629
batch 2900: loss 0.009506
batch 3000: loss 0.342969
batch 3100: loss 0.002595
batch 3200: loss 0.041201
batch 3300: loss 0.050962
batch 3400: loss 0.011064
batch 3500: loss 0.014094
batch 3600: loss 0.023903
batch 3700: loss 0.075372
batch 3800: loss 0.00172

# Evaluation

In [7]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.976900
